# ひき算
量子コンピュータには従来のコンピュータと同じ計算もできるという特徴があります。ひき算の回路を確認します。

## 今回学ぶこと
1. 量子ゲートを使って二進数のひき算を実装
2. 量子の重ね合わせを使って1つの回路で複数のひき算

## Blueqatのインストール
pipからBlueqatをインストールします。

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 二進数のひき算
たし算は符号を判定するccxゲートと、位の足し合わせを実行するcxゲートを使います。今回はa-b=cdという二進数のひき算の量子回路を行います。今回はaとbの値によって4種類のひき算を実装します。それぞれのひき算は、

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

となります。4量子ビットのビット列で表現し、前半の2量子ビットが入力値aとbで、後半の2量子ビットが出力値cとdです。aとbを入力する回路と、実際に引き算を実現する回路を別に作り、何度か使い回します。1のようなデータの入力はXゲートを使って0を反転させて実装します。

ひき算の回路部分はこのようになります。*はコントロールビットです。

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

aとbにXゲートを使ってデータを入れると、ccx回路で符号の判定、cx回路で位の引き算を行います。

In [1]:
#ツールの読み込み
from blueqat import Circuit

#ひき算部分を実現します。
substractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [2]:
#0-0
(Circuit() + substractor).m[:].run(shots=100)

Counter({'0000': 100})

In [3]:
#0-1
(Circuit().x[1] + substractor).m[:].run(shots=100)

Counter({'0111': 100})

In [5]:
#1-0
(Circuit().x[0] + substractor).m[:].run(shots=100)

Counter({'1001': 100})

In [6]:
#1-1
(Circuit().x[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 100})

このように、ひき算が実装できました。

## 重ね合わせを利用したひき算
ここで、Xゲートでデータを一つ一つ入れる代わりに、Hゲートを使ってひき算をしてみます。

In [7]:
#Xゲートの代わりにHゲートを入力に使う
(Circuit().h[0,1] + substractor).m[:].run(shots=100)

Counter({'0000': 25, '1100': 23, '1001': 25, '0111': 27})

アダマールゲートを利用すると4つのひき算が大体1/4ずつ答えとして出てきました。このように汎用ひき算回路を作ると、重ね合わせ状態を利用した計算を実行できます。

## もつれを利用したひき算
次にHゲートの代わりにa-b=0となるたし算を量子のもつれを使って行ってみます。

In [8]:
#00と11のもつれを作る
(Circuit().h[0].cx[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 51, '0000': 49})

このように、入力値が00と11がもつれているので、この2つのひき算が約1/2ずつ出てきます。

--------

## 解説：回路の作成（符号）
まずは上の符号から作成します。
符号は、0, 1, 0, 0 の順になっています。

表で表すと以下になります。

|X|Y|符号|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|0|
|1|1|0|

次にXの桁を反転させると

|X|Y|符号|
|:-:|:-:|:-:|
|1|0|0|
|1|1|1|
|0|0|0|
|0|1|0|

これを見ると CCXゲートと同じことがわかります。
(CCXは入力の頭2つのビットが1ならば3番目のビットを反転させる)

このことから入力が 11 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_0.png" width="23%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（２桁目）
次にもう一つのくらいを作成します。
2桁目を見ると、0, 1, 1, 0 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの1桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

これを見ると CXゲートと同じことがわかります。
(左辺がCXの入力で右辺が出力の2番目のビット)

このことから入力が 10 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_1.png" width="27%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（全体）
最後に上の二つの回路をまとめます。

入力を 00 としてまとめた回路を以下に示します。

<img src="./img/008/008_02_3.png" width="40%">

上2ビットを入力、残りの2ビットが出力です。
始めの CCX は符号の部分で残りの 2つの CX は引き算の部分です。   
符号を検知するためにXゲートを施したのでCCXした後は再びXをして戻しています。

## 解説：重ね合わせ回路の実装
量子の重ね合わせを用いると4パターンを一気に操作することができます。

式としては以下の物を考えます。

<img src="./img/009/009_02_0.png" width="35%">

見てわかる通り右辺に入力の4パターンが出てきました。
この性質を用いて実装します。

回路は以下のようになります。

<img src="./img/009/009_02_2.png" width="40%">

今まで Xゲートを施していた部分を Hゲートにして 4パターンを作っています。

## (応用) 一般の引き算
一般の10進数同士で引き算を実装します。  
引き算回路は足し算回路を逆にすることで実装できます。

足し算回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

右から回路を考えます。
$a, a+b$ を入力して $b$ を出力します。 $a = a_n ... a_0$, $(a+b) = b_n ... b_0$ と2進数で表示できます。  
(ここで n は数が大きい方の位を基準とします。)

$c_i$ は繰り下がりを表します。  
この回路は carry と呼ばれる繰り上がりの部分と sum の合計の部分にわかれています。
まずはこの2つの部分を考察します。

### 繰り下がり
carry, sum は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F00361854-e782-0da0-3386-f59f668a1ada.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=58d1d41b0893b16bfb24bfd546328ff2" width="60%">

回路の始めの部分はこれらを組み合わせて繰り下がりを求めています。

### オバーフロービット
$a < a+b$ の場合でしか引き算はできません。   
判別を $b_{n+1}$ で行います。これをオーバーフロービットと言います。

$a < a+b$ のとき $b_{n+1}$ = 0  
$a > a+b$ のとき $b_{n+1}$ = 1 になります。

$a > a+b$ のときは $2^{n+1} - b$ が出力されます。

### 実装
引き算回路を実装します。

流れは以下のようになります。  
1. carry の逆回路と sum 回路で各位の繰り下がりを求める。  
2. オーバーフローを判別して $b_{n+1}$ に格納する。  
3. carry 回路で差を出力する。    

以上の流れから $b$ は $b_n ... b_0$ の部分に出力されることが分かります。
$b_{n+1}$ はオーバーフロービットです。

実装するために下準備をします。  
これは足し算回路と同じなので説明は省略します。

In [3]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:]

def digits(a,b): 
     # 2進数に変換
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # nを決めて大きい方にビット数を合わせる
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # マッピング
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

def todecimal(A):
    return int(str(A),2) 

def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

### 一般化回路
以上で全ての準備が整ったので引き算回路を作ります。

In [12]:
def minus(a,ab): 
     # 入れ替え
     c = ab
     ab = a
     a = c

     # 2進数表示のマッピング
     qubits = len(digits(a,ab)) 
     cir1 = toX(digits(a,ab)) 
     digi = int((len(digits(a,ab))-1)/3) 

     # 前半のcarry回路とsum逆回路
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     # 最後の桁の処理
     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[-3,-2] 

     # carryの逆回路でbiに差を出力
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3)

     result = (cir1 + cir4 + cir3 + cir2).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

実際に計算してみます。

In [7]:
minus(8,2)

6

In [9]:
minus(4,2)

2

In [10]:
minus(50,24)

26

計算できました。ちなみに $a > a + b$ のときは

In [11]:
minus(2,4)

14

こちらもきちんと計算されています。

### 参考文献
https://arxiv.org/pdf/quant-ph/9511018.pdf